**BITPANDA IMPORT CSVDATA for DCA cost INFORMATION**

In [1]:
from modules import general_utils
from modules.general_utils import glob, os, np, pd, dt, go, sp, yf
from modules import collect_data_utils
from modules.collect_data_utils import get_current_user,collect_data_from_csv,collect_bitpanda_data,sort_data_by_asset,collect_data_from_list_csv,collect_numb_sample,collect_file

path = f"C:\\Users\\{get_current_user()}\\Downloads\\Telegram Desktop\\bitpanda-trades-2025-02-18-19-50.csv"


In [8]:
multiple_files = 0

# Assuming data_collected is a dictionary returned by collect_bitpanda_data
data_collected = collect_bitpanda_data(path)
data_collected = sort_data_by_asset(data_collected)
# Convert the relevant columns to numeric types
df = pd.DataFrame(data_collected)
df['amount_asset_collect'] = pd.to_numeric(df['amount_asset_collect'], errors='coerce')
df['amount_fiat_collect'] = pd.to_numeric(df['amount_fiat_collect'], errors='coerce')
df['asset_market_price_collect'] = pd.to_numeric(df['asset_market_price_collect'], errors='coerce')

# Filter out 'stake' and 'transfer' transactions
df = df[~df['trans_type_collect'].isin(['stake', 'transfer'])]

# Adjust the amounts for 'sell' transactions
df.loc[df['trans_type_collect'] == 'sell', 'amount_asset_collect'] *= -1

# Ignore 'deposit' entries for BTC
df = df[~((df['asset_collect'] == 'BTC') & (df['trans_type_collect'] == 'deposit'))]

# Group by 'asset_collect' and calculate the sum of 'amount_asset_collect' and 'asset_market_price_collect'
grouped = df.groupby('asset_collect').agg({
    'amount_asset_collect': 'sum',
    'amount_fiat_collect': 'sum'
}).reset_index()

# Calculate the median price for each asset
grouped['median_price'] = grouped['amount_fiat_collect'] / grouped['amount_asset_collect']

# Print the resulting DataFrame with the calculated sums and median prices
print(grouped)



  asset_collect  amount_asset_collect  amount_fiat_collect  median_price
0           BNB              0.000023                24.44  1.076652e+06
1           BTC              0.004956               557.28  1.124408e+05
2          DOGE            953.392548                 0.00  0.000000e+00
3           ETH              0.000008                86.50  1.034689e+07
4           EUR              0.000000               929.01           inf
5           SOL              4.044039               856.98  2.119119e+02
6         TRUMP              1.000000                68.62  6.862000e+01
7           XRP              0.007261                34.97  4.815836e+03


c:\Users\david\Documents\MyGITprj\modules\collect_data_utils.py:203: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  collected_data[key].append(row[column_index])
